In [2]:
import torch
import torchvision
from torchvision import transforms, datasets

train = datasets.MNIST('', train=True, download=True,transform=transforms.Compose([transforms.ToTensor()]))

test = datasets.MNIST('', train=False, download=True,transform=transforms.Compose([transforms.ToTensor()]))

0.0%Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to MNIST\raw\train-images-idx3-ubyte.gz
100.1%Extracting MNIST\raw\train-images-idx3-ubyte.gz to MNIST\raw
0.0%Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to MNIST\raw\train-labels-idx1-ubyte.gz
Extracting MNIST\raw\train-labels-idx1-ubyte.gz to MNIST\raw
180.4%Extracting MNIST\raw\t10k-images-idx3-ubyte.gz to MNIST\raw
Extracting MNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST\raw
Processing...
Done!


In [3]:
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False)

In [4]:
import torch.nn as nn
import torch.nn.functional as F 

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:
import torch.optim as optim

loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)

EPOCHS = 3

for epoch in range(EPOCHS):
    for data in trainset:
        X, y = data
        net.zero_grad()
        output = net(X.view(-1,28*28))
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
    print(loss)


tensor(0.1064, grad_fn=<NllLossBackward>)
tensor(0.0649, grad_fn=<NllLossBackward>)
tensor(0.0066, grad_fn=<NllLossBackward>)


In [7]:
for epoch in range(EPOCHS):
    for data in testset:
        X, y = data
        net.zero_grad()
        output = net(X.view(-1,28*28))
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
    print(loss)

tensor(0.0105, grad_fn=<NllLossBackward>)
tensor(0.0105, grad_fn=<NllLossBackward>)
tensor(0.0011, grad_fn=<NllLossBackward>)


In [26]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1, 28*28))
        correct += output.argmax(dim=1).eq(y).sum().item()
        total += len(y)

print("Accucracy: ", round(correct/total, 3))

Accucracy:  0.983


In [27]:
correct = 0
total = 0

with torch.no_grad():
    for data in trainset:
        X, y = data
        output = net(X.view(-1, 28*28))
        correct += output.argmax(dim=1).eq(y).sum().item()
        total += len(y)

print("Accucracy: ", round(correct/total, 3))

Accucracy:  0.967
